## Save activation values

In [3]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('donkeycar')
sys.path.append('src')


In [4]:
from tensorflow import keras
import multiprocessing
import numpy as np
from losses import get_pipe, get_dataset_from_pipe
from donkeycar.utils import get_model_by_type
from dataloader import load_records
import donkeycar as dk
import matplotlib.pyplot as plt


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [17]:
cfg = dk.load_config(config_path='../mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False
N_FOLDS = 5
DIRECTION = 'CC'
DATA_PATH = '../data/'

model_type = '3d-modified-steering'

loading config file: ../mycar/config.py
loading personal config over-rides from myconfig.py


In [11]:
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)


INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/faster-autodrive/catalog_20.catalog


19250

In [12]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/slower-autodrive/catalog_20.catalog


20304

In [13]:
fast_model = get_model_by_type(model_type, cfg)
fast_model.interpreter.model.summary()


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d_3 (Conv3D)           (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 1, 10, 26, 16)    0         
 3D)                                                             
                                                                 
 conv3d_4 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [54]:
from model_loader import load_model
from sklearn.model_selection import KFold
import math

kf = KFold(n_splits=N_FOLDS, shuffle=False)

fast_activations = np.zeros((len(fast_speed_data), 256))
slow_activations = []

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d'

for fold, (train_index, test_index) in enumerate(kf.split(fast_speed_data), start=1):
    model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('dense_1').output)

    test_records = [fast_speed_data[i] for i in test_index]
    cfg.BATCH_SIZE = max((math.gcd(len(test_records), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, test_records)
    data = get_dataset_from_pipe(pipe)
    fold_test_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    fast_activations[test_index, :] = fold_test_activations

    cfg.BATCH_SIZE = max((math.gcd(len(slow_speed_data), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, slow_speed_data)
    data = get_dataset_from_pipe(pipe)
    fold_slow_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    slow_activations.append(fold_slow_activations)


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


54/54 [==============================] - 39s 725ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


54/54 [==============================] - 38s 697ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


54/54 [==============================] - 37s 692ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


54/54 [==============================] - 34s 637ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


54/54 [==============================] - 34s 630ms/step


In [58]:
np.save("nighttime_ai_fast_model_fast_data_activations.npy", np.float16(fast_activations))
np.save("nighttime_ai_fast_model_slow_data_activations.npy", np.float16(slow_activations))

In [81]:

slow_activations = np.zeros((len(slow_speed_data), 256))
fast_activations = []

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d'

for fold, (train_index, test_index) in enumerate(kf.split(slow_speed_data), start=1):
    model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('dense_1').output)

    test_records = [slow_speed_data[i] for i in test_index]
    cfg.BATCH_SIZE = max((math.gcd(len(test_records), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, test_records)
    data = get_dataset_from_pipe(pipe)
    fold_test_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    slow_activations[test_index, :] = fold_test_activations

    cfg.BATCH_SIZE = max((math.gcd(len(fast_speed_data), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, fast_speed_data)
    data = get_dataset_from_pipe(pipe)
    fold_fast_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    fast_activations.append(fold_fast_activations)


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


31/31 [==============================] - 11s 325ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


31/31 [==============================] - 10s 317ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


31/31 [==============================] - 7s 234ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


31/31 [==============================] - 7s 228ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


14/14 [==============================] - 7s 488ms/step


In [82]:
np.save("nighttime_ai_slow_model_fast_data_activations.npy", np.float16(fast_activations))
np.save("nighttime_ai_slow_model_slow_data_activations.npy", np.float16(slow_activations))